In [20]:
import pandas as pd

gs_res = pd.read_csv('output/project2_grid_search.csv', index_col='Unnamed: 0').sort_values(by=['val_rmse'])
#0.829998
gs_res#[gs_res['cv']!=True]

,bagging_fraction,bagging_freq,bagging_seed,best_round,boosting,cv,feature_fraction,feature_fraction_seed,gs_timespent,learning_rate,max_depth,metric,min_data_in_leaf,num_leaves,num_threads,objective,scale_pos_weight,timestamp,val_rmse
1529337857,0.9,1,24742346,20,gbdt,True,1.0,24742346,0:00:00,0.10,-1,rmse,32,61,8,regression,2,2018-06-18 12:04:17,0.949705
1529337859,1.0,1,26731722,20,gbdt,True,0.9,26731722,0:00:00,0.03,-1,rmse,37,15,8,regression,2,2018-06-18 12:04:19,1.039360


In [13]:
gs_res.iloc[0]

bagging_fraction                                   1
bagging_freq                                       1
bagging_seed                                17078476
best_round                                       771
boosting                                        gbdt
categorical_column       "[4, 6, 9, 10, 15, 16, 19]"
cv                                              True
feature_fraction                                 0.4
feature_fraction_seed                       17078476
gs_timespent                                 0:00:26
learning_rate                                   0.03
max_depth                                         -1
metric                                          rmse
min_data_in_leaf                                  23
num_leaves                                        61
num_threads                                        8
objective                                 regression
scale_pos_weight                                   2
timestamp                        2018-06-15 22